In [4]:
import argparse

import torch
import torch.nn as nn
import torch.optim as optim

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Bitcoin_Trading')

from rnn import LSTM
from trainer import Trainer
from utils import load_data

In [25]:
def define_argparser():
  p=argparse.ArgumentParser()

  p.add_argument('--model_fn', required=True)
  p.add_argument('--gpu_id', type=int, default=0 if torch.cuda.is_available() else -1)

  p.add_argument('--batch_size', type=int, default=1)
  p.add_argument('--n_epochs', type=int, default=20)
  p.add_argument('--verbose', type=int, default=2)
  p.add_argument('--train_ratio', type=int, default=0.8)

  p.add_argument('--input_dim', type=int, default=5) # n_feature=train_x_array.shape[2], 특성의 개수
  p.add_argument('--hidden_dim', type=int, default=32)  
  p.add_argument('--output_dim', type=int, default=5)  # n_feature=train_x_array.shape[2], 특성의 개수
  p.add_argument('--num_layers', type=int, default=1)    
  p.add_argument('--lookback', type=int, default=40)   

  config=p.parse_args(["--model_fn", "True"])
  # default를 바꾸고 싶으면 config.gpu_id=0 코드를 써주어서 바꾸면 됨.

  return config




def main(config):
  device=torch.device('cpu') if config.gpu_id <0 else torch.device('cuda:%d' % config.gpu_id)

  x,y= load_data() ##(7362, 1380, 5), (7362, 120,5)를 가져오게 됨

  train_cnt=int(x.size(0)*config.train_ratio)
  valid_cnt=x.size(0)-train_cnt

  ## Shuffle dataset to split into train/valid set.
  ## index_select을 통해서 indices에 맞춰 배열해주고, split을 통해 [5362, 1380, 5], [2299, 1380, 5] 로 나누기
  indices = torch.randperm(x.size(0)) #tensor([6407, 1132, 3554,  ..., 5267, 5668, 3429])
  x=torch.index_select(x, dim=0, index=indices).to(device).split([train_cnt, valid_cnt],dim=0) # [5362, 1380, 5], [2299, 1380, 5] 
  y=torch.index_select(y, dim=0, index=indices).to(device).split([train_cnt, valid_cnt],dim=0) # [5362, 120, 5], [2299, 120, 5] 

  print("Train:", x[0].shape, y[0].shape)
  print("Valid:", x[1].shape, y[1].shape)
##################################################################################
  model  = LSTM(input_dim=config.input_dim, hidden_dim=config.hidden_dim, output_dim=config.output_dim, num_layers=config.num_layers).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  crit = torch.nn.MSELoss()

  trainer = Trainer(model, optimizer, crit)

  trainer.train((x[0], y[0]), (x[1], y[1]), config)

  # Save best model weights.
  torch.save({
      'model': trainer.model.state_dict(),
      'config': config,
  }, config.model_fn)

  PATH='/content/drive/MyDrive/Colab Notebooks/Bitcoin_Trading'

  torch.save({
      'model_state_dict' : trainer.model.state_dict(),
      'config' : config,

  }, PATH+'/final_model.pth')



In [ ]:
  if __name__ == '__main__':
    config = define_argparser()
    main(config)
